Few ideas so far:

## 1 FFNN based
- Train an nn on-top of sentence transformer to output logits / probs for score
  - optimise against given scores


## 2 LLM based
- Other approach might be to get a good language model to summarise the given prompt,
- embed its output
- embed the inference input
- get the absolute difference between the two, using the llm output as a ground truth
  - the closer the distance, the better the score we assign to the prompt
- could also experiment with fine-tuning the llm against the train dataset, or other summarisation datasets
  - would be a good opportunity to experiment with techniques such as peft and lora

## 3 ensemble
- Ensemble everything, maybe using somehing like rank averaging